# Position Prediction for Football Players
**by Ciutat Vella Team**: Tatiana Bakwenye, Natalia Beltran, Mikel Gallo

## 0. Set Up

#### Import Packages 

In [3]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import seaborn as sns
import os
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
#from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint
#from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# # import own modules
import function_file as fp
from function_file import * 

#### Import Data

In [4]:
# Retrieve path and load the data
path = os.getcwd()
dr = pd.read_csv(path+"/train.csv")
dt = pd.read_csv(path+"/test.csv")

# Number of rows and columns
print('dr train', dr.shape)
print('df test', dt.shape)

dt.head(3)

dr train (6895, 70)
df test (767, 69)


,id,short_name,overall,potential,value_eur,wage_eur,birthday_date,height_cm,weight_kg,club_name,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,215562,R. Hunt,63,63,475000.0,3000,1995-07-07,171,67,Swindon Town,...,57,54,59,58,15,12,12,11,15,NaN
1,248311,Tiri,64,64,500000.0,950,1991-07-14,186,84,ATK Mohun Bagan FC,...,55,63,63,64,9,14,9,11,6,NaN
2,223933,G. Nilsson,64,70,925000.0,2000,1997-05-23,197,87,SV Wehen Wiesbaden,...,55,23,25,19,6,7,8,16,14,NaN


#### Merge Train and Test Datasets

Create function for merging train and test datasets

In [5]:
# Create column to differentiate train from test
dr['is_test'] = 0
dt['is_test'] = 1

In [6]:
# union both datasets
df = pd.concat([dr, dt])

In [7]:
# Unit test
dr.shape[0] + dt.shape[0] == df.shape[0]

True

In [8]:
# Unit test
df['is_test'].value_counts()

is_test
0    6895
1     767
Name: count, dtype: int64

#### Data Variable Breakdown

#### player identification and overall qualification variables  
- id: Unique identifier for each player.
- short_name: Short name or nickname of the player. [X]
- birthday_date: Player's date of birth. [X]
- height_cm: Player's height in centimeters. [D]
- weight_kg: Player's weight in kilograms. [D]

- overall: Player's overall rating, representing their overall skill level [D]
- potential: Player's potential rating, indicating their potential skill growth. [D]
- international_reputation: Player's international reputation level. [D]
- club_name: Name of the player's club.[X]
- league_name: Name of the league the club belongs to.[D]
- league_level [D]
- club_jersey_number [D]
- club_loaned_from [X]
- club_joined [X]
- club_contract_valid_until [X]
- nationality_name [D]
- nation_jersey_number [X]

- work_rate [D]
- body_type [D]
- real_face [X]
- player_tags [X]
- player_traits [D]
  
#### player financial metrics  
- value_eur: Player's market value in euros.
- wage_eur: Player's weekly wage in euros.
- release_clause_eur

  
#### player position related metrics  
- position: Player's preferred playing position.[TARGET VARIABLE]
- preferred_foot: Player's preferred kicking foot (left or right).
- weak_foot: Player's weak foot rating, indicating their weaker kicking foot's ability.

#### Skills
- skill_dribbling
- skill_curve
- skill_fk_accuracy
- skill_long_passing
- skill_ball_control
- skill_moves: Player's skill moves rating, representing their dribbling and ball control skills.

#### high level attributes  
- pace 
- shooting 
- passing 
- dribbling 
- defending 
- physic: Attributes representing different aspects of a player's playing style and skills.

#### mentality attributes  
- mentality_aggression
- mentality_vision
- mentality_composure: Attributes representing mental aspects of a player's game.
- mentality_interceptions
- mentality_positioning
- mentality_penalties
   
#### attacking attributes  
- attacking_crossing
- attacking_finishing 
- attacking_heading_accuracy: Attributes related to attacking and finishing skills.'attacking_crossing', 
- attacking_heading_accuracy
- attacking_short_passing
- attacking_volleys

#### movement attributes 
- movement_acceleration
- movement_sprint_speed
- movement_agility: Attributes related to a player's speed and agility.
- movement_reactions
- movement_balance

#### strength attributes 
- power_shot_power
- power_jumping
- power_stamina: Attributes representing a player's physical power and endurance.
- power_strength
- power_long_shots

#### defending attributes
- defending_marking_awareness
- defending_standing_tackle
- defending_sliding_tackle: Attributes representing a player's defensive skills.


#### Goalkeeper attributes  

- goalkeeping_diving
- goalkeeping_handling
- goalkeeping_positioning: Goalkeeping attributes related to diving, handling, and positioning.
- goalkeeping_reflexes
- goalkeeping_speed: Attributes representing a goalkeeper's reflexes and speed.
- goalkeeping_kicking


## 1. Dataset Exploration

## 2. Handling Missing Data

#### Checking for Missing Values  

In [9]:
def missing_value(dataset):
    mis = dataset.isna().mean()
    return mis[mis > 0].sort_values(ascending = False)

In [10]:
missing_value(df)

club_loaned_from        0.948316
nation_jersey_number    0.939441
goalkeeping_speed       0.908510
player_tags             0.888280
player_traits           0.375098
position                0.100104
pace                    0.091490
shooting                0.091490
passing                 0.091490
dribbling               0.091490
defending               0.091490
physic                  0.091490
release_clause_eur      0.052336
club_joined             0.051684
value_eur               0.000653
dtype: float64

### NEED TO CHECK THINK WE DIDN'T DROP THE NATION JERSEY NUMBER

Can be dropped:
- club_loaned_from
- nation_jersey_number
- player_tags

Needs transformation: 
- goalkeeping_speed
- player_traits

##### Variable Dropping with High Missing Values 

In [ ]:
# Dropping the top three variables with a high percent of missing values 
columns_to_drop = ['club_loaned_from', 'nation_jersey_number', 'player_tags']
df = df.drop(columns=columns_to_drop)

##### Variable Transformation with NAs 

In [ ]:
# Goalkeeping speed
# Function to impute NA with 0 
def impute_column_with_zero(df, column_name):
    df[column_name] = df[column_name].fillna(0)
    return df

df = impute_column_with_zero(df, 'goalkeeping_speed')

### NEED TO CHECK ORDER CLEANS DATA THEN DOES VISUAL?

## 3. Feature Engineering 

### Create function for calculating players age

In [ ]:
# Calculate age of players
df['birthday_date'] = pd.to_datetime(df['birthday_date'], format='%Y-%m-%d', errors='coerce')
df['today'] = pd.to_datetime('2022-11-01',format='%Y-%m-%d', errors='coerce')

# Create the new age column 
df['age'] = ((df['today'] - df['birthday_date'])/np.timedelta64(1, 'Y'))
df['age'] = df['age'].astype(int)

### Create function for calculating years in the club 

In [ ]:
# Calculate the number of years in the club
df['club_joined'] = pd.to_datetime(df['club_joined'], format='%Y-%m-%d', errors='coerce')
df['today'] = pd.to_datetime('2022-11-01', format='%Y-%m-%d', errors='coerce')

# Calculate the years_in_club only if 'club_joined' is not NA
df['years_in_club'] = np.where(df['club_joined'].notna(), ((df['today'] - df['club_joined']) / np.timedelta64(1, 'Y')), np.nan)
df['years_in_club'] = df['years_in_club'].astype(float).round(2)

# Drop the 'club_joined' column
df = df.drop('club_joined', axis=1)

### Create function for clustering positions into goalkeeper, defender, midfielder and attacker

df.position.unique()

In [ ]:
# Creating high level position categories
conditions = [
    df['position'].isin(['GK']),
     df['position'].isin(['LB','LCB','CB','RCB','RB','LWB','LDM','CDM','RDM','RWB']),
     df['position'].isin(['LM','LCM','CM','RCM','RM']),
     df['position'].isin(['LAM','CAM','RAM','LW','LF','CF','RF','RW','LS','ST','RS'])
     
]

choices = ['Goalkeeper', 'Defense', 'Midfield', 'Attack']

df['job'] = np.select(conditions, choices, default='other')

In [ ]:
# Creating high level position
conditions = [
    df['position'].isin(['GK']),
     df['position'].isin(['LB','LCB','CB','RCB','RB']),
     df['position'].isin(['LWB','LDM','CDM','RDM','RWB']),
     df['position'].isin(['LM','LCM','CM','RCM','RM']),
     df['position'].isin(['LAM','CAM','RAM']),
     df['position'].isin(['LW','LF','CF','RF','RW']),
     df['position'].isin(['LS','ST','RS'])
     
]

choices = ['GK', 'CB', 'CDM', 'CM','CAM','CF','ST']

df['position_line'] = np.select(conditions, choices, default='other')

In [ ]:
# Creating high position location
conditions = [
    df['position'].isin(['GK','CB','CDM','CM','CAM','CF','ST']),
     df['position'].isin(['RCB','RDM','RCM','RAM','RF','RS']),
     df['position'].isin(['LCB','LDM','LCM','LAM','LF','LS']),
     df['position'].isin(['RB','RWB','RM','RW']),
     df['position'].isin(['LB','LWB','LM','LW'])
     
]

choices = ['CC', 'CR', 'CL', 'R','L']

df['position_loc'] = np.select(conditions, choices, default='other')

In [ ]:
# Creating high position location
conditions = [
    df['position'].isin(['LB','LWB','LM','LW','RB','RWB','RM','RW'])
     
]

choices = ['yes']

df['is_wing'] = np.select(conditions, choices, default='no')

In [ ]:
# Creating high level position
conditions = [
    df['position'].isin(['GK']),
     df['position'].isin(['LB','LCB','CB','RCB','RB']),
     df['position'].isin(['LWB','LDM','CDM','RDM','RWB']),
     df['position'].isin(['LM','LCM','CM','RCM','RM']),
     df['position'].isin(['LAM','CAM','RAM']),
     df['position'].isin(['LW','LF','CF','RF','RW']),
     df['position'].isin(['LS','ST','RS'])
     
]

choices = [0,1,2,3,4,5,6]

df['position_line_ord'] = np.select(conditions, choices, default=-1)

In [ ]:
conditions = [
    df['position'].isin(['GK']),
     df['position'].isin(['LB','LCB','CB','RCB','RB','LWB','LDM','CDM','RDM','RWB']),
     df['position'].isin(['LM','LCM','CM','RCM','RM']),
     df['position'].isin(['LAM','CAM','RAM','LW','LF','CF','RF','RW','LS','ST','RS'])
     
]

choices = [0,1,2,3]

df['job_order'] = np.select(conditions, choices, default=-1)

In [ ]:
# extract the side of the player from the position L or R
df.loc[:, "side"] = df.loc[:, "position"].str[0]

### Create a copy of df without GK

In [14]:
dk = df[(df['position']!= 'GK')&(df['is_test']== 0)]

In [15]:
# Unit test
print(df.shape)
print(dk.shape)

(7662, 71)
(6264, 71)


## Train & Test Models 

## Final Model

## Conclusion